In [33]:
import pandas as pd

In [38]:
df = pd.read_excel('~/Downloads/metro_station.xlsx')  

In [39]:
df['coordinates'] = df['lon'].astype(str) + ','+ df['lng'].astype(str)
#df['coordinates1'] = df['Широта1'].astype(str) + ','+ df['Долгота1'].astype(str)


In [41]:
df["timeBetweenStop"] = df["timeBetweenStop"] * 60

In [37]:
df[["Вестибюль станции", "Следующая станция", "coordinates0"]]

,Вестибюль станции,Следующая станция,coordinates0
0,Проспект Ветеранов,Ленинский проспект,"59.84188,30.251543"
1,Ленинский проспект,Автово,"59.851677,30.268279"
2,Автово,Кировский завод,"59.867369,30.261345"
3,Кировский завод,Нарвская,"59.879726,30.261908"
4,Нарвская,Балтийская,"59.901169,30.274676"
...,...,...,...
63,Адмиралтейская,Спортивная-1,"59.935877,30.314886"
64,Спортивная-1,Чкаловская,"59.952078,30.291312"
65,Чкаловская,Крестовский остров,"59.961035,30.291964"
66,Крестовский остров,Старая Деревня,"59.971838,30.259427"


In [ ]:
# Dictionary of edges, indexed in both directions by node number.
edge = {}

# Ingest the data and build teh dictionary
with open("edges.txt") as efile:
    for line in efile:
        eid, src, dst = line.strip().split(';')
        src = int(src)
        dst = int(dst)

        for key, val in [(src, dst), (dst, src)] :
            if key in edge:
                edge[key].append(val)
            else:
                edge[key] = ([val])
print "edge dictionary has entries:", len(edge)

# Identify endpoint nodes: order other than 2
end_ct = 0
print "Endpoint Nodes"
endpoint = []
for src, dst in edge.iteritems():
    if len(dst) != 2:
        print len(dst), src, dst
        endpoint.append(src)
        end_ct += len(dst)
print end_ct, "road ends"

atlas = []    # List of roads, each a list of nodes

# Build roads between the identified endpoints
# Pick the first endpoint in the remaining list.
# Move to the first-listed adjacent node.
# Keep going until we hit another node on the endpoint list.
while len(endpoint) > 0:
    here = endpoint[0]
#   print "Road starting at", here, edge[here]

    # Pick a first step and consume the edge
    next = edge[here].pop(0)
    edge[next].remove(here)
    road = [here, next]

    # If that's the last connection to the node, remove that node from the endpoints list.
    if len(edge[here]) == 0:
        del endpoint[0]
        del edge[here]
    # Special case for a one-segment road; endpoint entry of "next" is removed after the loop
    if len(edge[next]) == 0:
        del edge[next]

    # Consume edges until we reach another endpoint.
    debug = False
    while next not in endpoint:
        here = next
        next = edge[here].pop(0)
        edge[next].remove(here)
        road.append(next)
        if len(edge[next]) == 0:
            del edge[next]
#           print "removing node", next

    if next not in edge:
        endpoint.remove(next)
#       print "removing endpoint", next

    print "\nRoad from", road[0], "to", road[-1], ':\n\t', road
    atlas.append(road)

print "\n", len(atlas), "roads built"
# print "edge dictionary still has entries:", len(edge)